# A Practical Guide to Activation Threshold Search in Post-Training Quantization

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/mct_features_notebooks/keras/example_keras_activation_threshold_search.ipynb)

## Overview
This tutorial demonstrates how to find the optimal activation threshold, a key component in MCT's post-training quantization workflow.

In this example, we will explore two different metrics for threshold selection. We will begin by applying the appropriate MCT configurations, followed by inferring a representative dataset through the model. Next, we will plot the activation distributions of two layers along with their corresponding MCT-calculated thresholds, and finally, we will compare the quantized model accuracy using both methods.

## Activation threshold explanation
During the quantization process, thresholds are used to map a distribution of 32-bit floating-point values to their quantized equivalents. Minimizing data loss while preserving the most representative range is crucial for maintaining the final model's accuracy.

### How Is It Done in MCT?

MCT's post-training quantization leverages a representative dataset to evaluate a range of typical output activation values. The challenge lies in determining the best way to map these values to their quantized versions. To address this, a grid search is performed to find the optimal threshold using various error metrics. Typically, mean squared error (MSE) is the most effective and is used as the default metric.

The error is calculated based on the difference between the original float and the quantized distributions. The optimal threshold is then selected based on the metric that results in the minimum error. For example, for the case of MSE.

$$
ERR(t) = \frac{1}{n_s} \sum_{X \in Fl(D)} (Q(X, t, n_b) - X)^2
$$

- $ERR(t)$ : The quantization error function dependent on the threshold $t$.
- 
- $n_s$: The size of the representative dataset.

- $\sum$: Summation over all elements $X$ in the flattened dataset $F_l(D)$.

- $F_l(D)$: The set of activation tensors in the $l$-th layer, flattened for processing.

- $Q(X, t, n_b)$: The quantized approximation of $X$, given a threshold $t$ and bit width $n_b$.

- $X$: The original activation tensor before quantization.

- $t$: The quantization threshold, a key parameter for controlling the quantization process.

- $n_b$: The number of bits used in quantization, impacting model precision and size.


Quantization thresholds often have specific limitations, typically imposed for deployment purposes. In MCT, activation thresholds are restricted by default to Power-of-Two values and can represent either signed values within the range $(-T, T)$ or unsigned values within $(0, T)$. Other restriction settings are also configurable.

### Error methods supported by MCT:

- **NOCLIPPING:** Use min/max values as thresholds.

- **MSE:** Minimizes quantization noise by using the mean squared error (MSE).

- **MAE:** Minimizes quantization noise by using the mean absolute error (MAE).

- **KL:** Uses Kullback-Leibler (KL) divergence to align the distributions, ensuring that the quantized distribution is as similar as possible to the original.

- **Lp:** Minimizes quantization noise using the Lp norm, where `p` is a configurable parameter that determines the type of distance metric.

## Setup
Install the relevant packages:

In [ ]:
TF_VER = '2.14'
!pip install -q tensorflow[and-cuda]~={TF_VER}

In [ ]:
import importlib
if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

In [ ]:
import keras
import tensorflow as tf

Load a pre-trained MobileNetV2 model from Keras, in 32-bits floating-point precision format.

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

float_model = MobileNetV2()

## Dataset preparation
### Download the ImageNet validation set
Download the ImageNet dataset with only the validation split.
**Note:** For demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [ ]:
import os
 
if not os.path.isdir('imagenet'):
    !mkdir imagenet
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_devkit_t12.tar.gz
    !wget -P imagenet https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    
    !cd imagenet && tar -xzf ILSVRC2012_devkit_t12.tar.gz && \
     mkdir ILSVRC2012_img_val && tar -xf ILSVRC2012_img_val.tar -C ILSVRC2012_img_val

The following code organizes the extracted data into separate folders for each label, making it compatible with Keras dataset loaders.

In [ ]:
from pathlib import Path
import shutil

root = Path('./imagenet')
imgs_dir = root / 'ILSVRC2012_img_val'
target_dir = root /'val'

def extract_labels():
    !pip install -q scipy
    import scipy
    mat = scipy.io.loadmat(root / 'ILSVRC2012_devkit_t12/data/meta.mat', squeeze_me=True)
    cls_to_nid = {s[0]: s[1] for i, s in enumerate(mat['synsets']) if s[4] == 0} 
    with open(root / 'ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt', 'r') as f:
        return [cls_to_nid[int(cls)] for cls in f.readlines()]

if not target_dir.exists():
    labels = extract_labels()
    for lbl in set(labels):
        os.makedirs(target_dir / lbl)
    
    for img_file, lbl in zip(sorted(os.listdir(imgs_dir)), labels):
        shutil.move(imgs_dir / img_file, target_dir / lbl)

These functions generate a `tf.data.Dataset` from image files in a directory.

In [ ]:
def imagenet_preprocess_input(images, labels):
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

def get_dataset(batch_size, shuffle):
    dataset = tf.keras.utils.image_dataset_from_directory(
        directory='./imagenet/val',
        batch_size=batch_size,
        image_size=[224, 224],
        shuffle=shuffle,
        crop_to_aspect_ratio=True,
        interpolation='bilinear')
    dataset = dataset.map(lambda x, y: (imagenet_preprocess_input(x, y)), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

## Representative Dataset
For quantization with MCT, we need to define a representative dataset required by the PTQ algorithm. This dataset is a generator that returns a list of images:

In [ ]:
batch_size = 32
n_iter = 10

dataset = get_dataset(batch_size, shuffle=True)

def representative_dataset_gen():
    for _ in range(n_iter):
        yield [dataset.take(1).get_single_element()[0].numpy()]

## Target Platform Capabilities
MCT optimizes the model for dedicated hardware. This is done using TPC (for more details, please visit our [documentation](https://sony.github.io/model_optimization/docs/api/api_docs/modules/target_platform.html)). Here, we use the default Tensorflow TPC:

In [ ]:
import model_compression_toolkit as mct

# Get a TargetPlatformCapabilities object that models the hardware for the quantized model inference. Here, for example, we use the default platform that is attached to a Keras layers representation.
target_platform_cap = mct.get_target_platform_capabilities('tensorflow', 'default')

## Post-Training Quantization using MCT
In this step, we load the model and apply post-training quantization using two threshold error calculation methods: **"No Clipping"** and **MSE**.

- **"No Clipping"** selects the lowest power-of-two threshold that ensures no data is lost (clipped).
- **MSE** selects a power-of-two threshold that minimizes the mean square error between the original float distribution and the quantized distribution.

- As a result, the "No Clipping" method typically results in a larger threshold, as we will demonstrate later in this tutorial.

The quantization parameters are predefined, and we use the default values except for the quantization method. Feel free to modify the code below to experiment with other error metrics supported by MCT.

In [ ]:
from model_compression_toolkit.core import QuantizationErrorMethod

q_configs_dict = {}
# Error methods to iterate over
error_methods = [
    QuantizationErrorMethod.MSE,
    QuantizationErrorMethod.NOCLIPPING
]

# If you are curious you can add any of the below quantization methods as well.
# QuantizationErrorMethod.MAE
# QuantizationErrorMethod.KL
# QuantizationErrorMethod.LP

# Iterate and build the QuantizationConfig objects
for error_method in error_methods:
    q_config = mct.core.QuantizationConfig(
        activation_error_method=error_method,
    )

    q_configs_dict[error_method] = q_config

Now we will run post-training quantization for each configuration:

In [ ]:
quantized_models_dict = {}

for error_method, q_config in q_configs_dict.items():
    # Create a CoreConfig object with the current quantization configuration
    ptq_config = mct.core.CoreConfig(quantization_config=q_config)

    # Perform MCT post-training quantization
    quantized_model, quantization_info = mct.ptq.keras_post_training_quantization(
        in_model=float_model,
        representative_data_gen=representative_dataset_gen,
        core_config=ptq_config,
        target_platform_capabilities=target_platform_cap
    )

    # Update the dictionary to include the quantized model
    quantized_models_dict[error_method] = {
        "quantization_config": q_config,
        "quantized_model": quantized_model,
        "quantization_info": quantization_info
    }


## Threshold and Distribution Visualization
To facilitate understanding, we will plot the activation distributions for two layers of MobileNetV2. For each layer, we will show the thresholds determined by both **MSE** and **No Clipping** methods, along with the corresponding activation distributions obtained by infering the representative dataset through the model. This visualization highlights the trade-off between data loss and data resolution under different thresholds during quantization.

MCT’s `quantization_info` stores the threshold values for each layer. However, to view the actual activation distributions, the model needs to be reconstructed up to and including the target layer selected for visualization.

To do this, we first need to identify the layer names. In Keras, this can be easily done for the first 10 layers using the following code snippet.

In [ ]:
for index, layer in enumerate(float_model.layers):
    if index < 10:
        print(layer.name)
    else:
        break

The first activation layer in the model is named `Conv1_relu`.

For this particular model, testing has shown that the `expanded_conv_project_BN` layer exhibits different thresholds for the two error metrics. Therefore, we will also include this layer in the visualization. For context, MobileNetV2 uses an inverted residual structure, where the input is first expanded in the channel dimension, then passed through a depthwise convolution, and finally projected back to a lower dimension. The `expanded_conv_project_BN` layer represents this projection, and the BN suffix indicates the presence of Batch Normalization.

We will use these layer names to create two separate models, each ending at one of these respective layers.

In [ ]:
from tensorflow.keras.models import Model
layer_name1 = 'Conv1_relu'
layer_name2 = 'expanded_conv_project_BN'

layer_output1 = float_model.get_layer(layer_name1).output
activation_model_relu = Model(inputs=float_model.input, outputs=layer_output1)
layer_output2 = float_model.get_layer(layer_name2).output
activation_model_project = Model(inputs=float_model.input, outputs=layer_output2)

Infer the representative dataset using these models and store the outputs for further analysis.

In [ ]:
import numpy as np
activation_batches_relu = []
activation_batches_project = []
for images in representative_dataset_gen():
    activations_relu = activation_model_relu.predict(images)
    activation_batches_relu.append(activations_relu)
    activations_project = activation_model_project.predict(images)
    activation_batches_project.append(activations_project)

all_activations_relu = np.concatenate(activation_batches_relu, axis=0).flatten()
all_activations_project = np.concatenate(activation_batches_project, axis=0).flatten()

Thresholds calculated by MCT during quantization can be accessed using the following approach. The layer indices correspond to the order of the layers listed in the previous steps.

As noted earlier, we focus on the first ReLU activation layer and the Batch Normalization layer (`expanded_conv_project_BN`) since they effectively illustrate the impact of the two threshold error methods.

In [ ]:
# layer 4 is the first activation layer - Conv1_relu
layer_name2 = 'expanded_conv_project_BN'
optimal_thresholds_relu = {
    error_method: data["quantized_model"].layers[4].activation_holder_quantizer.get_config()['threshold'][0]
    for error_method, data in quantized_models_dict.items()
}

# layer 9 is the batch normalisation projection layer - Expanded_conv_project_BN
optimal_thresholds_project = {
    error_method: data["quantized_model"].layers[9].activation_holder_quantizer.get_config()['threshold'][0]
    for error_method, data in quantized_models_dict.items()
}

### Distribution Plots
Below are the activation distributions for the two selected layers: first, the ReLU activation layer, `Conv1_relu`, followed by the `expanded_conv_project_BN` layer.

The second distribution clearly highlights the differences between the two error metrics, showing the impact of each on the resulting quantization threshold.

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(10, 6))
plt.hist(all_activations_relu, bins=100, alpha=0.5, label='Original')
for method, threshold in optimal_thresholds_relu.items():
    plt.axvline(threshold, linestyle='--', linewidth=2, label=f'{method}: {threshold:.2f}')

plt.title('Activation Distribution with Optimal Quantization Thresholds First Relu Layer')
plt.xlabel('Activation Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(10, 6))
plt.hist(all_activations_project, bins=100, alpha=0.5, label='Original')
for method, threshold in optimal_thresholds_project.items():
    plt.axvline(threshold, linestyle='--', linewidth=2, label=f'{method}: {threshold:.2f}')

plt.title('Activation Distribution with Optimal Quantization Thresholds Project BN layer')
plt.xlabel('Activation Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## Model Evaluation
Finally, we can demonstrate the impact of these different thresholds on the model's overall accuracy.
In order to evaluate our models, we first need to load the validation dataset.

In [ ]:
val_dataset = get_dataset(batch_size=50, shuffle=False)

In [ ]:
float_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics="accuracy")
float_accuracy = float_model.evaluate(val_dataset)
print(f"Float model's Top 1 accuracy on the Imagenet validation set: {(float_accuracy[1] * 100):.2f}%")

In [ ]:
evaluation_results = {}

for error_method, data in quantized_models_dict.items():
    quantized_model = data["quantized_model"]

    quantized_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

    results = quantized_model.evaluate(val_dataset, verbose=0)  # Set verbose=0 to suppress the log messages

    evaluation_results[error_method] = results

    # Print the results
    print(f"Results for {error_method}: Loss = {results[0]}, Accuracy = {results[1]}")

These results are consistent across many models, which is why MSE is set as the default method.

Each of MCT's error methods impacts models differently, so it is recommended to include this metric as part of hyperparameter tuning when optimizing quantized model accuracy.


## Conclusion
In this tutorial, we explored the process of finding optimal activation thresholds using different error metrics in MCT’s post-training quantization workflow. By comparing the **MSE** and **No Clipping** methods, we demonstrated how the choice of threshold can significantly affect the activation distributions and, ultimately, the quantized model’s performance. While **MSE** is commonly the best choice and is used by default, it is essential to consider other error metrics during hyperparameter tuning to achieve the best results for different models.

Understanding the impact of these thresholds on data loss and resolution is critical when fine-tuning the quantization process for deployment, making this a valuable step in building high-performance quantized models.


## Appendix
Below is a code snippet that can be used to extract information from each layer in the MCT quantization output, assisting in analyzing the layer-wise quantization details.

In [ ]:
import tensorflow as tf

quantized_model = data["quantized_model"]
quantizer_object = quantized_model.layers[1]

quantized_model = data["quantized_model"]


relu_layer_indices = []


for i, layer in enumerate(quantized_model.layers):
    # Convert the layer's configuration to a string
    layer_config_str = str(layer.get_config())

    layer_class_str = str(layer.__class__.__name__)

    # Check if "relu" is mentioned in the layer's configuration or class name
    if 'relu' in layer_config_str.lower() or 'relu' in layer_class_str.lower():
        relu_layer_indices.append(i)

print("Layer indices potentially using ReLU:", relu_layer_indices)
print("Number of relu layers " + str(len(relu_layer_indices)))


In [ ]:
for error_method, data in quantized_models_dict.items():
    quantized_model = data["quantized_model"]
    print(quantized_model.layers[1])

Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
